<a href="https://colab.research.google.com/github/0seoyeon/tobigs-21st/blob/%EA%B3%B5%EC%84%9C%EC%97%B0/Week5/KNN%26Clustering/%EA%B3%B5%EC%84%9C%EC%97%B0/Tobigs_Week5_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 21기 KNN 정규세션 과제

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# 현재 작업 디렉토리 확인
current_directory = os.getcwd()
print("Current directory:", current_directory) # /content

# 작업 디렉토리 내의 파일 목록 확인
# os.listdir(current_directory)

# 작업 디렉토리 변경
new_directory = os.chdir('/content/drive/MyDrive/Tobigs/regular_session')
current_directory = os.getcwd()
print('Current directory:', current_directory)

Mounted at /content/drive
Current directory: /content
Current directory: /content/drive/MyDrive/Tobigs/regular_session


In [ ]:
import numpy as np
import pandas as pd
import math

## 0. 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv("./data/blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA

### Exploration

In [ ]:
df.info() # missing values at Product_Category_2 and Product_Category_3

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
User_ID,4998.0,1.003015e+06,1709.909665,1000003.0,1001560.00,1003040.0,1004416.75,1006037.0
Occupation,4998.0,8.036815e+00,6.442697,0.0,2.00,7.0,14.00,20.0
Marital_Status,4998.0,4.213685e-01,0.493828,0.0,0.00,0.0,1.00,1.0
Product_Category_1,4998.0,5.320128e+00,3.912281,1.0,1.00,5.0,8.00,20.0
Product_Category_2,3465.0,9.773737e+00,5.101867,2.0,5.00,9.0,15.00,18.0
Product_Category_3,1544.0,1.276684e+01,4.023591,3.0,9.00,14.0,16.00,18.0
Purchase,4998.0,9.234655e+03,4999.225081,13.0,5497.75,8049.0,12039.00,23913.0


In [ ]:
# Distinguish Categorical values and Numerical values
cat_df = df.select_dtypes(include='object')
num_df = df.select_dtypes(exclude='object')
cat_df.columns, num_df.columns

(Index(['Product_ID', 'Gender', 'Age', 'City_Category',
        'Stay_In_Current_City_Years'],
       dtype='object'),
 Index(['User_ID', 'Occupation', 'Marital_Status', 'Product_Category_1',
        'Product_Category_2', 'Product_Category_3', 'Purchase'],
       dtype='object'))

In [ ]:
print(len(df['User_ID'].unique())) # 2656
print(len(df['Product_ID'].unique())) # 1797

2656
1797


In [ ]:
df = df.drop(columns=['User_ID', 'Product_ID'])

### Scaling & Remove Outliers

In [ ]:
def Zscore(data): # normalization
  # z_score 구하기
  mean = np.mean(data)
  std = np.std(data)
  z_scores = (data - mean) / std

  return z_scores

num_df = Zscore(num_df)
num_df.head()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,-1.126917,0.304745,-0.853356,-0.081835,1.416600,NaN,-1.445300
1,0.864615,0.304745,-0.853356,0.429427,NaN,NaN,-0.950574
2,1.337787,0.304745,-0.853356,-1.104358,-1.131854,NaN,-0.307810
3,-0.974847,0.304745,-0.853356,-0.848727,-1.131854,NaN,1.439236
4,-0.974847,0.304745,-0.853356,-0.593097,-0.935819,NaN,-0.691107


In [ ]:
# Extract instances where Outliers by Zscore exists (# threshold = 3)
Zscore_outliers = num_df[(num_df > 3) | (num_df < -3)]
Zscore_outliers = Zscore_outliers.dropna(how='all')
Zscore_outliers.info() # 69 outliers at Product_Category_1

# Remove the rows with Outliers by Z_score
df = df.drop(index=Zscore_outliers.index)

# Check the result
df.info() # 4998 -> 4929

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 26 to 4965
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User_ID             0 non-null      float64
 1   Occupation          0 non-null      float64
 2   Marital_Status      0 non-null      float64
 3   Product_Category_1  69 non-null     float64
 4   Product_Category_2  0 non-null      float64
 5   Product_Category_3  0 non-null      float64
 6   Purchase            0 non-null      float64
dtypes: float64(7)
memory usage: 4.3 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4929 entries, 0 to 4997
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      4929 non-null   object 
 1   Age                         4929 non-null   object 
 2   Occupation                  4929 non-null   int64  
 3   City_Category    

### One-Hot Encoding

In [ ]:
df = pd.get_dummies(df)

# check the result
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4929 entries, 0 to 4997
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Occupation                     4929 non-null   int64  
 1   Marital_Status                 4929 non-null   int64  
 2   Product_Category_1             4929 non-null   int64  
 3   Product_Category_2             3465 non-null   float64
 4   Product_Category_3             1544 non-null   float64
 5   Purchase                       4929 non-null   int64  
 6   Gender_F                       4929 non-null   uint8  
 7   Gender_M                       4929 non-null   uint8  
 8   Age_0-17                       4929 non-null   uint8  
 9   Age_18-25                      4929 non-null   uint8  
 10  Age_26-35                      4929 non-null   uint8  
 11  Age_36-45                      4929 non-null   uint8  
 12  Age_46-50                      4929 non-null   u

In [ ]:
# Replace missing values at Product_Category_2
from sklearn.linear_model import LinearRegression

cat2_test = df[df['Product_Category_2'].isnull()] # Nan to replace
cat2_train =  df.drop(index=cat2_test.index)

cat2_X_train = cat2_train.drop(columns = ['Product_Category_2', 'Product_Category_3'])
cat2_y_train = cat2_train['Product_Category_2']

cat2_X_test = cat2_test.drop(columns = ['Product_Category_2', 'Product_Category_3'])
cat2_y_test = cat2_test['Product_Category_2']

lr = LinearRegression()
lr2 = lr.fit(cat2_X_train, cat2_y_train)
cat2_predict = lr2.predict(cat2_X_test)
df.loc[cat2_test.index, 'Product_Category_2'] = cat2_predict

# check the result
df.loc[cat2_test.index] # filled with predicted values

,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,...,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
1,10,0,7,12.246750,NaN,4483,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
5,10,0,8,13.113761,NaN,9821,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
8,10,0,16,19.634385,NaN,16521,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
10,10,0,1,7.310095,NaN,18964,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
15,0,0,8,12.689435,NaN,9902,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4974,8,1,5,10.820432,NaN,7163,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
4985,3,0,12,16.211852,NaN,1039,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
4987,6,1,5,11.016117,NaN,5368,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
4994,1,1,8,13.287950,NaN,2045,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0


In [ ]:
# Replace missing values at Product_Category_3
cat3_test = df[df['Product_Category_3'].isnull()] # Nan to replace
cat3_train =  df.drop(index=cat3_test.index)

cat3_X_train = cat3_train.drop(columns = ['Product_Category_3'])
cat3_y_train = cat3_train['Product_Category_3']

cat3_X_test = cat3_test.drop(columns = ['Product_Category_3'])
cat3_y_test = cat3_test['Product_Category_3']

lr3 = lr.fit(cat3_X_train, cat3_y_train)
cat3_predict = lr3.predict(cat3_X_test)
df.loc[cat3_test.index, 'Product_Category_3'] = cat3_predict

# check the result
df.loc[cat3_test.index]

,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,...,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
0,10,0,5,17.000000,16.952937,2010,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,10,0,7,12.246750,14.193211,4483,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
2,10,0,1,4.000000,10.492784,7696,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,10,0,2,4.000000,11.261624,16429,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
4,10,0,3,5.000000,11.272254,5780,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,7,1,5,14.000000,16.709585,5310,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
4993,7,1,1,16.000000,18.115295,11701,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
4994,1,1,8,13.287950,16.138468,2045,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
4996,0,1,12,16.453624,17.309380,1413,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0


## 2. KNN 구현 & 파라미터 튜닝

- KNN Regression for 'Purchase'

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.3, random_state = 42)

In [ ]:
X_train = train.drop(columns=['Purchase'])
y_train = train['Purchase'] # value to predict: Purchase

X_test = test.drop(columns=['Purchase'])
y_test = test['Purchase']

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
k_lst = [3, 5, 10, 20, 50]
pred_lst = []
for k in k_lst:
  knn = KNeighborsRegressor(n_neighbors=k)
  knn.fit(X_train, y_train) # train
  pred = knn.predict(X_test) # predict
  print(f'result with k={k}: {pred}')
  pred_lst.append(pred)


result with k=3: [ 6369.33333333 13023.33333333  9307.         ... 18301.33333333
  1321.33333333  5806.        ]
result with k=5: [ 6281.2 12467.6  9542.  ... 15055.   4108.6  6315.2]
result with k=10: [ 5968.3 12085.6  8566.  ... 14463.7  4496.3  5970.1]
result with k=20: [ 6615.25 12236.25  6882.4  ... 12546.1   3667.55  7145.75]
result with k=50: [ 6770.18 13068.92  7454.12 ... 12708.38  5286.2   7420.7 ]


## 3. Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for i in range(len(k_lst)):
  # 평균 제곱 오차(MSE) 계산
  mse = mean_squared_error(y_test, pred_lst[i])
  print(f"MSE with k={k_lst[i]}:", mse)

  # 평균 절대 오차(MAE) 계산
  mae = mean_absolute_error(y_test, pred_lst[i])
  print(f"MAE with k={k_lst[i]}:", mae)

  # R² 스코어 계산
  r2 = r2_score(y_test, pred_lst[i])
  print(f"R² Score with k={k_lst[i]}:", r2)
  print()

MSE with k=3: 17296999.121478476
MAE with k=3: 3110.4119900833894
R² Score with k=3: 0.30991104223731714

MSE with k=5: 16272691.765814738
MAE with k=5: 3027.141176470588
R² Score with k=5: 0.3507772751910375

MSE with k=10: 15611770.929344151
MAE with k=10: 2953.4934415145367
R² Score with k=10: 0.37714567400984056

MSE with k=20: 15977501.829318797
MAE with k=20: 2976.5442528735634
R² Score with k=20: 0.36255430739112393

MSE with k=50: 16517535.496861663
MAE with k=50: 2992.3834212305615
R² Score with k=50: 0.34100887814214786



k=10일 때의 성능이 가장 좋으나, 전반적으로 성능이 매우 낮다.